### get real prompt

In [ ]:
from pycocotools.coco import COCO

coco = COCO('/mnt/d/data/learning/ML/defake/mscoco/annotations/captions_train2014.json')
imgId_list = coco.getImgIds()
imgId_list.sort()

from tqdm import tqdm
with open('/mnt/d/data/learning/ML/defake/mscoco/real_prompts.txt', 'w') as f:
	for imgId in tqdm(imgId_list):
		annIds = coco.getAnnIds(imgId)
		caption = coco.loadAnns(annIds)[0]['caption'].strip() # select first caption
		caption = caption.replace('\n', ' ')
		f.write(caption.strip() + '\n')
  
with open('/mnt/d/data/learning/ML/defake/mscoco/real_prompts.txt', 'r') as f:
    lines = f.readlines()
    
len(lines)

### generate prompt with real image

In [ ]:
import os
import torch
from PIL import Image
from natsort import natsorted
from transformers import BlipProcessor, BlipForConditionalGeneration
from tqdm import tqdm

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base", torch_dtype=torch.float16).to("cuda")
image_dir = '/mnt/d/data/learning/ML/defake/mscoco/train2014' # <- 改路徑
image_filenames = natsorted(os.listdir(image_dir))

with open('/mnt/d/data/learning/ML/defake/mscoco/real_img_fake_prompts.txt', 'w') as f: # <- 改路徑
	for filename in tqdm(image_filenames):
		raw_image = Image.open(os.path.join(image_dir, filename)).convert('RGB')
		inputs = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)
		out = model.generate(**inputs)
		caption = processor.decode(out[0], skip_special_tokens=True)
		f.write(caption + '\n')


### generate prompt with fake image

In [ ]:
import os
import torch
from PIL import Image
from natsort import natsorted
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base", torch_dtype=torch.float16).to("cuda")
image_dir = '/mnt/d/data/learning/ML/defake/mscoco/train2014' # <- 改路徑
image_filenames = natsorted(os.listdir(image_dir))

with open('/mnt/d/data/learning/ML/defake/mscoco/fake_img_fake_prompts.txt', 'w') as f: # # <- 改路徑
	for filename in image_filenames:
		raw_image = Image.open(os.path.join(image_dir, filename)).convert('RGB')
		inputs = processor(raw_image, return_tensors="pt").to("cuda", torch.float16)
		out = model.generate(**inputs)
		caption = processor.decode(out[0], skip_special_tokens=True)
		f.write(caption + '\n')